In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatText
from IPython.display import display, Math

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

## Warm-Up Problems

Consider the following problems as we wait for class to begin.

# Day 18: Determinants

<font color="red"> Return to this notebook and include some discussion on the three ways to compute a determinant: (i) as the "volume" of a geometric object, (ii) through cofactor expansion, which is inefficient, but useful in some cases, and (iii) using row-reduction to obtain an upper triangular matrix and computing the determinant from there, which is much more efficient than cofactor expansion.</font>

While we can go through with the row reduction process to determine whether a matrix is invertible or not, it would be helpful if we could obtain a scalar that would provide us with this information. Scalars are much easier to work with than matrices.

In this notebook, we'll be introduced to the notion of the *determinant* of a matrix. We'll see that the determinant provides us with a scalar encoding whether or not a matrix is invertible.

### Motivation

Use the interactive textboxes below to investigate the area of of the parallelogram whose sides are determined by the column vectors in the matrix $A$. The format for the matrix is $A = \begin{bmatrix} \texttt{a1} & \texttt{b1}\\ \texttt{a2} & \texttt{b2}\end{bmatrix}$. Experiment with mutliple different vectors and structure for the matrix $A$. Before you finish, be sure to examine what happens if you choose two vectors that are *not* linearly independent.

In [ ]:
# @title
def plot_parallelogram(a1, a2, b1, b2):
    A = np.array([[a1, b1],
                  [a2, b2]])
    origin = np.array([[0, 0], [0, 0]])
    vectors = np.array([[a1, a2], [b1, b2]])

    A_mat = sp.Matrix(A)
    latex_string = r"\large{"  + "A = " + sp.latex(A_mat) + "}"
    display(Math(latex_string))

    fig, ax = plt.subplots(figsize=(6,6))
    ax.quiver(*origin, vectors[:, 0], vectors[:, 1], angles='xy', scale_units='xy', scale=1, color=['r','b'])

    # Draw parallelogram
    p1 = [0, 0]
    p2 = [a1, a2]
    p3 = [a1 + b1, a2 + b2]
    p4 = [b1, b2]
    parallelogram = np.array([p1, p2, p3, p4, p1])
    ax.plot(parallelogram[:, 0], parallelogram[:, 1], 'k--')

    det = np.linalg.det(A)
    ax.set_title(f"Area = {abs(det):.2f}")
    ax.set_xlim(min(-5, a1 + b1) - 1, max(5, a1 + b1) + 1)
    ax.set_ylim(min(-5, a2 + b2) - 1, max(5, a2 + b2) + 1)
    ax.axhline(y=0, color='black')
    ax.axvline(x=0, color='black')
    ax.set_aspect('equal')
    plt.grid(True)
    plt.show()

interact(plot_parallelogram,
         a1=FloatText(min=-10, max=10, value=1),
         a2=FloatText(min=-10, max=10, value=0),
         b1=FloatText(min=-10, max=10, value=0),
         b2=FloatText(min=-10, max=10, value=1))


interactive(children=(FloatText(value=1.0, description='a1'), FloatText(value=0.0, description='a2'), FloatTex…

<function __main__.plot_parallelogram(a1, a2, b1, b2)>

Now check out a similar interactive plot below, but with vectors in $\mathbb{R}^3$. This time, we'll be interested in calculating the volume of the parallelepiped whose sides are determined by the column vectors of the matrix $A$.

In [ ]:
# @title
def plot_parallelepiped_3D(a1, a2, a3, b1, b2, b3, c1, c2, c3):
    # Define vectors
    a = np.array([a1, a2, a3])
    b = np.array([b1, b2, b3])
    c = np.array([c1, c2, c3])
    A = np.column_stack((a, b, c))

    # Display matrix
    A_mat = sp.Matrix(A)
    latex_string = r"\large{" + "A = " + sp.latex(A_mat) + "}"
    display(Math(latex_string))

    # Compute volume using determinant
    volume = abs(np.linalg.det(A))

    # Plotting
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    origin = np.array([0, 0, 0])
    ax.quiver(*origin, *a, color='r', label='a', arrow_length_ratio=0.05)
    ax.quiver(*origin, *b, color='b', label='b', arrow_length_ratio=0.05)
    ax.quiver(*origin, *c, color='g', label='c', arrow_length_ratio=0.05)

    # Compute corners of the parallelepiped
    p0 = origin
    p1 = a
    p2 = b
    p3 = a + b
    p4 = c
    p5 = a + c
    p6 = b + c
    p7 = a + b + c

    verts = [
        [p0, p1, p3, p2],  # bottom
        [p0, p1, p5, p4],  # side a-c
        [p0, p2, p6, p4],  # side b-c
        [p7, p6, p4, p5],  # top
        [p7, p6, p2, p3],  # side b-a
        [p7, p5, p1, p3],  # side a-b
    ]

    # Dashed edges (for visibility of structure)
    edges = [
        (p0, p1), (p0, p2), (p0, p4),
        (p1, p3), (p1, p5),
        (p2, p3), (p2, p6),
        (p3, p7),
        (p4, p5), (p4, p6),
        (p5, p7),
        (p6, p7)
    ]

    for edge in edges:
        x_vals = [edge[0][0], edge[1][0]]
        y_vals = [edge[0][1], edge[1][1]]
        z_vals = [edge[0][2], edge[1][2]]
        ax.plot(x_vals, y_vals, z_vals, 'k--', linewidth=1)

    poly = Poly3DCollection(verts, alpha=0.3, facecolor='gray')
    ax.add_collection3d(poly)

    ax.set_title(f"Volume = {volume:.2f}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.set_xlim([-10, 10])
    ax.set_ylim([-10, 10])
    ax.set_zlim([-10, 10])
    ax.legend()
    plt.tight_layout()
    plt.show()

interact(plot_parallelepiped_3D,
         a1=FloatText(value=1), a2=FloatText(value=0), a3=FloatText(value=0),
         b1=FloatText(value=0), b2=FloatText(value=1), b3=FloatText(value=0),
         c1=FloatText(value=0), c2=FloatText(value=0), c3=FloatText(value=1))


interactive(children=(FloatText(value=1.0, description='a1'), FloatText(value=0.0, description='a2'), FloatTex…

<function __main__.plot_parallelepiped_3D(a1, a2, a3, b1, b2, b3, c1, c2, c3)>

Again, be sure to experiment with a variety of vectors. Examine sets of vectors $\vec{a}$, $\vec{b}$, and $\vec{c}$ that are *linearly independent* and also those that are *linearly dependent*. Check out the calculated volume as you do so.

> **Summary:** The area of a parallelogram formed by two two-dimensional vectors, the volume of a parallelepiped formed by three three-dimensional vectors, and their higher dimensional analogs can be used to identify whether or not a matrix with those vectors as its columns is invertible. If the column vectors are not linearly independent, then the dimension of the shape formed by those column vectors collapses and the corresponding measure (area, volume, etc.) will be $0$. In this case, the matrix is not invertible.

Now we just need to know how to calculate that area, volume, etc.! Determinants will do this for us (which we claim without proof). One caveat though is that it is possible to have a negative determinant. This is the case if pairs of vectors are *negatively oriented*, meaning that the angle measured counterclockwise between consecutive column vectors $\vec{v_i}$ and $\vec{v_{i+1}}$ is greater than $180^\circ$.

### Calculating Determinants

We saw the definition of a *determinant* of a $2\times 2$ matrix in our Day 16 notebook which introduced the notion of invertibility.

**Recall (Inverse of a $2\times 2$ Matrix):** Let $A = \left[\begin{array}{rr} a & b\\ c & d\end{array}\right]$, then as long as $\det\left(A\right) = ad - bc$ is non-zero, we have that $A$ is invertible and $\displaystyle{A^{-1} = \frac{1}{ad - bc}\left[\begin{array}{rr} d & -b\\ -c & a\end{array}\right]}$.

We'll add to this by showing how to compute the determinant of a square matrix which is larger than $2\times 2$, but for now, the definition of the determinant of a $2\times 2$ matrix is reiterated below.

**Definition (Determinant of a $2\times 2$ Matrix):** Consider the matrix $A = \left[\begin{array}{rr} a & b\\ c & d\end{array}\right]$. We have $\det\left(A\right) = ad - bc$.

**Completed Example 1:** Find the determinant of the matrix $A = \left[\begin{array}{rr} 2 & -6\\ 1 & 5\end{array}\right]$.

> *Solution.* We'll use the determinant formula above.
>
>\begin{align*} \det\left(\left[\begin{array}{rr} 2 & -6\\ 1 & 5\end{array}\right]\right) &= 2\left(5\right) - \left(-6\right)\left(1\right)\\
&= 10 + 6\\
&= 16 ~_\blacktriangledown
\end{align*}

**Definition ($ij$-Cofactor of $A$):** Let $A$ be an $n\times n$ matrix. We define the $ij$-cofactor of $A$, denoted by $A_{ij}$ to be the $\left(n-1\right) \times \left(n-1\right)$ matrix obtained from $A$ by deleting `row i` and `column j`.

**Definition (Determinant of an $n\times n$ Matrix):** Let $A$ be an $n\times n$ matrix. We can compute $\det\left(A\right)$ using cofactor expansion along any row, $i$. That is,

\begin{align*}\det\left(A\right) &= \sum_{j = 1}^{n}{\left(-1\right)^{i+j}a_{ij}\det\left(A_{ij}\right)}\\
&= \left(-1\right)^{i + 1}a_{i1}\det\left(A_{i1}\right) + \left(-1\right)^{i + 2}a_{i2}\det\left(A_{i2}\right) + \cdots + \left(-1\right)^{i + n}a_{in}\det\left(A_{in}\right)
\end{align*}

The above definition is recursive. That is, if $A_{ij}$ is not a $2\times 2$ matrix, we can use cofactor expansion to determine the determinant of $A_{ij}$. Computing the determinant of a large matrix requires good organization and book-keeping. We'll largely leave that process to computers. However, in order to get comfortable with the process of computing determinants, we'll compute determinants of $2\times 2$, $3\times 3$, and some $4\times 4$ matrices by hand. We'll also compute the determinants of larger matrices by hand if they have convenient structure.

**Completed Example 2:** Compute the determinant of the matrix $A = \left[\begin{array}{rrr} 1 & -2 & 5\\ 0 & 4 & 1\\ 1 & 2 & -1\end{array}\right]$.

> *Solution.* We'll use the cofactor expansion described above.
>
> \begin{align*} \det\left(\left[\begin{array}{rrr} 1 & -2 & 5\\ 0 & 4 & 1\\ 1 & 2 & -1\end{array}\right]\right) &= 1\det\left(\left[\begin{array}{rr} 4 & 1\\ 2 & -1\end{array}\right]\right) - \left(-2\right)\det\left(\left[\begin{array}{rr} 0 & 1\\ 1 & -1\end{array}\right]\right) + 5\det\left(\left[\begin{array}{rr} 0 & 4\\ 1 & 2\end{array}\right]\right)\\
&= \left(4\left(-1\right) - 1\left(2\right)\right) + 2\left(0\left(-1\right) - 1\left(1\right)\right) + 5\left(0\left(2\right) - 4\left(1\right)\right)\\
&= -6 - 2 - 20\\
&= -28 ~ _\blacktriangledown
\end{align*}
>
> *Note*. We could have saved a bit of work by expanding along the first column or the second row, taking advantage of the $0$ element in the matrix. We would end up with the same determinant -- we just need to take care in determining the signs on the terms in the cofactor expansion.

**Note (Feasibility of Computing Determinants):** At the beginning of our semester, we discussed that linear systems modeling real-world systems could easily utilize hundreds or thousands of variables and have hundreds or thousands of constraint equations. Even considering a $25\times 25$ matrix, a computer performing a trillion multiplications per second would take half a million years to compute its determinant using cofactor expansion! Fortunately there are faster methods, some of which exploit the structure of a matrix. One of those methods appears below.

**Strategy (Determinants of Triangular Matrices):** If $A$ is a triangular matrix, then $\det\left(A\right)$ is the product of the entries along the main diagonal of $A$.

**Completed Example 3:** Find the determinant of the matrix $A = \left[\begin{array}{rrrr} 2 & 1 & -4 & 8\\ 0 & -1 & 8 & 3\\ 0 & 0 & -3 & 0\\ 0 & 0 & 0 & 4\end{array}\right]$.

> *Solution.* Since the matrix is an upper-triangular matrix, its determinant is the product of its diagonal elements. That is, $\det\left(A\right) = 2\left(-1\right)\left(-3\right)\left(4\right) = 24$. $_\blacktriangledown$

**Notation:** Before we move forward, it is useful to mention some common notation. When stating that we are computing the determinant of a matrix, it is common to replace the brackets on either end of the matrix by vertical bars. For example, instead of writing $\det\left(\left[\begin{array}{rrr} a & b & c\\ d & e & f\\ g & h & i\end{array}\right]\right)$, it is common to write $\left|\begin{array}{rrr} a & b & c\\ d & e & f\\ g & h & i\end{array}\right|$ instead.  

## Try It!

The following examples will serve as classwork.

**Try It! 1:** Use the fact that if $A = \left[\begin{array}{rr} a & b\\ c & d\end{array}\right]$, then $\det\left(A\right) = ad - bc$ to compute the determinants of the following $2\times 2$ matrices.

$$A = \left[\begin{array}{rr} 2 & 6\\ -3 & 5\end{array}\right] ~~~~~~~ B = \left[\begin{array}{rr} -1 & 3\\ 8 & 2\end{array}\right]$$

> *Solution.*

**Try It! 2:** Use cofactor expansion to compute the determinants of the following $3\times 3$ matrices.

$$A = \left[\begin{array}{rrr} 1 & 2 & -1\\ -3 & 0 & 5\\ 4 & 2 & -1\end{array}\right] ~~~~~ B = \left[\begin{array}{rrr} 2 & -5 & 4\\ 0 & 1 & 0\\ 1 & 1 & 1\end{array}\right] ~~~~~ C = \left[\begin{array}{rrr} 3 & -1 & -1\\ 1 & 2 & 4\\ -3 & -1 & -2\end{array}\right]$$

> *Solution.*

**Try It! 3:** Compute the determinants of the following triangular matrices.

$$A = \left[\begin{array}{rr} 2 & 9\\ 0 & 3\end{array}\right] ~~~~~ B = \left[\begin{array}{rrr} -1 & 0 & 0 \\ 2 & 3 & 0\\ 3 & -3 & -5\end{array}\right] ~~~~~ C = \left[\begin{array}{rrrrr} 2 & 0 & 1 & 8 & 1\\ 0 & 1 & 0 & 0 & 0\\ 0 & 0 & 4 & 8 & -2\\ 0 & 0 & 0 & 3 & -1\\ 0 & 0 & 0 & 0 & 2\end{array}\right]$$

> *Solution.*

**Try It! 4:** Use strategic choices about cofactor expansion to compute the determinants of the following matrices.

$$A = \left[\begin{array}{rrrr} 1 & 2 & -5 & 1\\ -3 & 1 & 8 & 1\\ 0 & 2 & 0 & 0\\ 0 & -1 & 0 & 1\end{array}\right] ~~~~~~ B = \left[\begin{array}{rrrrr} 2 & -1 & 3 & 6 & 2\\ 0 & 0 & 5 & 0 & 0\\ 0 & -3 & 1 & 4 & -2\\ 0 & 0 & 2 & 1 & 7\\ 0 & 0 & 3 & 0 & 2\end{array}\right]$$

> *Solution.*

**Try It! 5:** Determine the impact of a row-swap operation on a $2\times 2$ matrix by finding $\det\left(\left[\begin{array}{rr} a & b\\ c & d\end{array}\right]\right)$ and $\det\left(\left[\begin{array}{rr} c & d\\ a & b\end{array}\right]\right)$.

> *Solution.*

**Try It! 6:** Determine the impact of scaling a row of a $2\times 2$ matrix by a constant $k$ by comparing $\det\left(\left[\begin{array}{rr} a & b\\ kc & kd\end{array}\right]\right)$ to $\det\left(\left[\begin{array}{rr} a & b\\ c & d\end{array}\right]\right)$.

> *Solution.*

**Try It! 7:** Determine the impact on the determinant of a $2\times 2$ matrix if we add a scalar multiple of the second row to the first row by comparing $\det\left(\left[\begin{array}{rr} a + kc & b + kd\\ b & c\end{array}\right]\right)$ to $\det\left(\left[\begin{array}{rr} a & b\\ b & c\end{array}\right]\right)$.

> *Solution.*

**Try It! 8:** Consider the unit square, whose corners are at $\vec{x_1} = \left[\begin{array}{r} 0\\ 0\end{array}\right]$, $\vec{x_2} = \left[\begin{array}{r} 1\\ 0\end{array}\right]$, $\vec{x_3} = \left[\begin{array}{r} 1\\ 1\end{array}\right]$, $\vec{x_4} = \left[\begin{array}{r} 0\\ 1\end{array}\right]$. Consider the matrix $A = \left[\begin{array}{rr} 2 & 0\\ 0 & 2\end{array}\right]$, and let $T\left(\vec{x}\right) = A\vec{x}$. Compute the transformations of the four vertices of the unit square, which will result in vertices of a new square. Compare the area of the new square to the area of the unit square. Now compute $\det\left(A\right)$ and compare that result to the ratio of the area of the new square to the area of the original.

Define another matrix $B = \left[\begin{array}{rr} 2 & -3\\ 1 & 2\end{array}\right]$. Let $T_2\left(\vec{x}\right) = B\vec{x}$. Use $\det\left(B\right)$ to determine the volume of the polygon whose vertices are the transformed positions of the vertices of the unit square under $T_2$.

> *Solution.*